# Пайплайн с полным пересчётом и записью только малой матрицы

In [39]:
import hail as hl

In [ ]:
hl.init()
hl.default_reference('GRCh38')

In [3]:
import glob
import os

In [ ]:
# конфигурация

# конфигурация
VCF_DIR = '/vcfs/'  # папка с VCF
VCF_DIR_NEW = '/new/' # папка с VCF для добавления
SEX_TABLE_PATH = 'sids_test.csv' # файл с полом
AF_PATH = '/vcfs/genomes/cache/af.tsv' # кеш частот

In [14]:
# старые файлы

vcf_files = glob.glob(VCF_DIR + '*.vcf.gz')
print(vcf_files)

['/vcfs/genomes/original/000007023790.vcf.gz', '/vcfs/genomes/original/000007023800.vcf.gz', '/vcfs/genomes/original/000007023820.vcf.gz', '/vcfs/genomes/original/000007023830.vcf.gz', '/vcfs/genomes/original/000007023840.vcf.gz', '/vcfs/genomes/original/000007023850.vcf.gz', '/vcfs/genomes/original/000007023860.vcf.gz', '/vcfs/genomes/original/000007023870.vcf.gz', '/vcfs/genomes/original/000007023880.vcf.gz', '/vcfs/genomes/original/000007023890.vcf.gz', '/vcfs/genomes/original/000007023910.vcf.gz', '/vcfs/genomes/original/000007023920.vcf.gz', '/vcfs/genomes/original/000007023930.vcf.gz', '/vcfs/genomes/original/000007023940.vcf.gz', '/vcfs/genomes/original/000007023950.vcf.gz', '/vcfs/genomes/original/000007023960.vcf.gz', '/vcfs/genomes/original/000007023970.vcf.gz', '/vcfs/genomes/original/000007023980.vcf.gz', '/vcfs/genomes/original/000007023990.vcf.gz', '/vcfs/genomes/original/000007024010.vcf.gz', '/vcfs/genomes/original/000007024020.vcf.gz', '/vcfs/genomes/original/000007024

In [15]:
# новые файлы
new_vcf_files = glob.glob(VCF_DIR_NEW + '*.vcf.gz')
print(new_vcf_files)

['/vcfs/genomes/new/000007029530.vcf.gz', '/vcfs/genomes/new/000007029510.vcf.gz', '/vcfs/genomes/new/000007029520.vcf.gz', '/vcfs/genomes/new/000007030390.vcf.gz']


In [7]:
# добавление пола

def set_sex(mt, sex_table):
    # преобразуем пол в is_female (True для 'ж'/'f')
    sex_table = sex_table.annotate(
        is_female = (
            (sex_table.sex.lower() == 'ж') | 
            (sex_table.sex.lower() == 'f')
        )
    )
    
    # добавляем is_female к образцам (простое соединение)
    mt = mt.annotate_cols(
        is_female = sex_table[mt.s].is_female  # mt.s - ID образца
    )

    return mt

In [8]:
# нормализация гемизигот у мужчин и МХ у всех

def normalize_ploidy(mt):
    return mt.annotate_entries(
        GT = hl.case()
            # Митохондриальная ДНК (гаплоидная у всех)
            .when(mt.locus.contig == "chrM",
                hl.if_else(
                    mt.GT.is_hom_ref(),
                    hl.call(0),
                    hl.if_else(
                        mt.GT.is_hom_var(),
                        hl.call(1),
                        hl.if_else(
                            (mt.VAF[0] > 0.3) | (mt.VAF[1] > 0.3),  # Учитываем оба аллеля
                            hl.call(1),
                            hl.call(0)
                        )
                    )
                )
            )
            # Гемизиготные участки у мужчин (X/Y)
            .when((~mt.is_female) & ((mt.locus.contig == "chrX") | (mt.locus.contig == "chrY")),
                hl.if_else(
                    mt.GT.is_hom_ref(),
                    hl.call(0),
                    hl.if_else(
                        mt.GT.is_hom_var(),
                        hl.call(1),
                        hl.if_else(
                            mt.VAF[0] > 0.3,
                            hl.call(1),
                            hl.call(0)
                        )
                    )
                )
            )
            # Все остальные случаи (аутосомы, X у женщин)
            .default(mt.GT)
    )

In [9]:
# фильтрация по глубине

def filter_variants_by_DP(combined_mt_all, dp):

    # отсекаем варианты, если нет ни одного образца с DP больше порога
    filtered_mt = combined_mt_all.filter_rows(
        hl.agg.count_where(
            (hl.is_defined(combined_mt_all.DP)) & 
            (combined_mt_all.DP >= dp)
        ) >= 1
    )

    # корректируем генотипы - варианты с DP меньше порога исключаем из расчёта частот, помечая как NA
    return filtered_mt.annotate_entries(
        GT = hl.if_else(
            (hl.is_defined(filtered_mt.DP)) & 
            (filtered_mt.DP >= dp),
            filtered_mt.GT,
            hl.missing(hl.tcall)
        )
    )


In [ ]:
#препроцессинг до фильтрации включительно

def preprocessing(vcf_files, sex_table):
    # комбайн
    mts_all = []
    for vcf in vcf_files: 
        mt = hl.import_vcf(vcf, force_bgz=True, array_elements_required=False)
        mt = set_sex(mt, sex_table)
        mt = normalize_ploidy(mt)
        mts_all.append(mt)

    # Объединение MatrixTable по колонкам (образцам)
    combined_mt_all = mts_all[0]
    if len(mts_all) > 1:
        for mt in mts_all[1:]:
            combined_mt_all = combined_mt_all.union_cols(mt, row_join_type='outer')

    #фильтрация по глубине
    return filter_variants_by_DP(combined_mt_all, 3)

In [ ]:
# расчёт частот

def mt_AF_calculated(mt):
    freq_mt_all = mt.annotate_rows(
    call_stats=hl.agg.call_stats(mt.GT, mt.alleles)
    hwe_stats=hl.agg.hardy_weinberg_test(mt.GT)
    )

    # извлечение частот аллелей
    return freq_mt_all.annotate_rows(
        allele_frequencies=freq_mt_all.call_stats.AF  # AF — это массив частот аллелей, включая мультиаллели
    )

## Пайплайн с добавлением новых данных - отсюда и до конца

In [16]:
# определение пола
sex_table = hl.import_table(SEX_TABLE_PATH,
        delimiter=',',
        types={'ID': hl.tstr, 'sex': hl.tstr},
        key='ID'
    )

In [ ]:
%%time

# препроцессинг данных
data = preprocessing(vcf_files, sex_table)

In [ ]:
%%time

# расчёт частот
data_af = mt_AF_calculated(data)

CPU times: user 15.8 ms, sys: 513 μs, total: 16.3 ms
Wall time: 15.1 ms


In [21]:
def make_af_table(data_af):
    filtered_mt = data_af.select_entries('DP', 'AD') \
                   .select_rows('rsid', 'allele_frequencies')
    return filtered_mt

In [ ]:
%%time

af = make_af_table(data_af)

CPU times: user 14.1 ms, sys: 0 ns, total: 14.1 ms
Wall time: 13.7 ms


In [ ]:
hl.stop()